## Notebook for the Graph Neural Network model
Multidisciplinary CBL 4CBLW00-01

How do we go with the flow?

Group 1  

Firstly, we import the necessary dataset containing the trajectories.

In [32]:
!pip install -q sociophysicsDataHandler
from sociophysicsDataHandler import SociophysicsDataHandler
dh = SociophysicsDataHandler()

You should consider upgrading via the 'C:\Users\karol\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


# Some theory
*A graph is a set of vertices connected to each other via edges. Formally we say that a graph G = (V,E), where V is the set of vertices and E is the set of edges.*

So, basically we want to make a model on these graphs, by encoding the entities, such as people and obstacles as vertices and connect them via edges so the model learns the relations between them. Each node will have a matrix with its features, and so will each edge. The network will then use them to learn how people behave and give us a prediction.

For now though, we must first make our graphs.

We import the necessary Graph Neural Network (GNN) libraries. (Some may not be used yet), I nevertheless decided to include them here.

In [ ]:
import numpy as np
import torch
import torch_cluster
import os
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader, Dataset
from torch_geometric.nn import global_mean_pool, HypergraphConv
from torch_cluster import knn_graph, radius_graph
from torch_geometric.utils import add_self_loops
from sklearn.model_selection import train_test_split
from scipy.spatial import cKDTree
from collections import defaultdict
from torch_geometric.nn import GCNConv, TransformerConv
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import degree

In [42]:
#Set device to cpu or cuda (nvidia graphics card) if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#We sort the dataframe by tracked_object and date_time_utc to ensure that the data is in the correct order for graph construction
#We also create a new column 'prev_x' and 'prev_y' to store the previous x and y positions of each tracked object

def add_dx_dy_ntype_city(df):
    """
    Adds displacement columns to the dataframe.
    """
    df = df.sort_values(['tracked_object', 'date_time_utc'])
    df['prev_x'] = df.groupby('tracked_object')['x_pos'].shift(1)
    df['prev_y'] = df.groupby('tracked_object')['y_pos'].shift(1)

    # Calculate displacements
    df['dx'] = (df['x_pos'] - df['prev_x']).fillna(0.0)
    df['dy'] = (df['y_pos'] - df['prev_y']).fillna(0.0)
    df['node_type'] = 0 # All nodes are of type 0 (human) for now
    df['city'] = 'EHV' # Add city column for later use
    return df

# We add a node type column - all nodes will be of type 1 as we only have humans now
# We will later add nodes corresponding to train doors, escalators and obstacles

#We add displacement columns for the displacement from the previous position to the current position; at the initial position, we set the displacement to 0.0
# # All nodes are of type 0 (human) for now
# Add city column for later use
#I was thinking we would want different radii for the radius-based edges: A train door affects people from a much larger distance
#psychologically than another person and we want to encode that relationship

#Looking at the graph for eindhhoven, 1m is about 880 pixels
#So we can set the radii for the different node types as follows:
RADII = {1: 2*880, 2: 10*880, 3: 5*880, 4: 5*880, 5: 5*880, 6: 3*880}
# which corresponds to
# 1: human, 2: train door, 3: escalator up, 4: escalator down, 5: stairs, 6: obstacle

#Also, to normalize our time later on, we need the minimum and maximum date we have data for
#For Eindhoven, we have data from:
# Midnight of April 1, 2021: 1617235200000 miliseconds since Jan 1, 1970
# to 
# Midnight of March 14, 2024: 1710374400000
# For utrecht:
# Midnight of March 22, 2020: 1584835200000

# Midnight of December 30, 2020: 1609286400000
# Hence we set as global variables:
min_time_ehv = 1617235200000
max_time_ehv = 1710374400000
min_time_utrecht = 1584835200000
max_time_utrecht = 1609286400000

min_y = -9000
max_y = 4500

min_x = -6000
max_x = 76000


Firstly, we make a funciton that makes a sine-cosine time encoding. Essentially the trick is to spread out the time variables assigned to each node as much as possible - the greater the differences, the better the model will learn time dependencies (so for example people behaving differently during the pandemic, or during peak hours and so on)

In [43]:
def sincos_time_embed(time, num_nodes, d_tau=8):
    half_dim = d_tau // 2
    k = torch.arange(half_dim, dtype=torch.float32, device=device)
    divisor = torch.pow(10000.0, (2 * k) / d_tau)
    args = time / divisor

    sin_emb = torch.sin(args)
    cos_emb = torch.cos(args)

    # Interleave sin and cos components
    emb = torch.empty(d_tau, dtype=torch.float32, device=device)
    emb[0::2] = sin_emb
    emb[1::2] = cos_emb

    # Repeat for all nodes
    return emb.unsqueeze(0).expand(num_nodes, -1)

In [ ]:
#We define the function human_graph to create a graph from the dataframe
def human_graph(df, time, radii=RADII):

    nodeid = torch.tensor(df['tracked_object'].values, dtype=torch.float, device=device).view(-1,1)
    x_pos_unn = torch.tensor(df['x_pos'].values, dtype=torch.float, device=device).view(-1,1) # Coordinates; unn stands for unnormalized
    y_pos_unn = torch.tensor(df['y_pos'].values, dtype=torch.float, device=device).view(-1,1)

    # Below we normalize time to [0,1]; 
    # this is important for the model to learn more efficiently - it will see a distrution of time in [0,1] instead of massive numbers;
    # in that sense it sees that the distribution is more close to the 2020s and not a uniform time block of 50 years
    # Note that some papers DO normalize coordinates as well, but I am not sure if this is necessary so I am not doing that just yet

    # dx = torch.tensor((df['dx'].values - mean_x) / std_x, dtype=torch.float, device=device).view(-1,1) # Displacements
    # dy = torch.tensor((df['dy'].values - mean_y) / std_y, dtype=torch.float, device=device).view(-1,1)

    dx = torch.tensor((df['dx'].values), dtype=torch.float, device=device).view(-1,1) # Displacements
    dy = torch.tensor((df['dy'].values), dtype=torch.float, device=device).view(-1,1)

    # Stack the features together
    type_idx = torch.tensor(df['node_type'].values, dtype=torch.long, device=device)
    N = x_pos_unn.size(0)
    type_onehot = F.one_hot(type_idx, num_classes=6).to(torch.float).to(device) #basically a vector of 1s and 0s, where the index of the 1 is the type of node
    # So for example if the node is of type 1, the vector will be [0,1,0], type 0 will be [1,0,0] and type 2 will be [0,0,1]

    tau   = sincos_time_embed(time, N)       # shape: [N, 8]
    node_feats = torch.cat([nodeid, x_pos_unn, y_pos_unn, dx, dy, type_onehot, tau], dim=1) # We concatenate the features together


    # 2) build edges
    # Use only spatial coords for connectivity
    pos = torch.cat([x_pos_unn, y_pos_unn], dim=1).to(device)

    # We add a radius graph to connect nodes that are within a certain distance of each other
    # I decided this would be a more appropriate choice than a knn graph
    
    # We make the radius edges ON CPU BECAUSE GPU IS BROKEN FOR MY GRAPHICS CARD
    pos_cpu = pos.detach().cpu()
    edge_index_cpu = radius_graph(pos_cpu, r=radii[1], loop=False)
    # Move edges back to GPU
    edge_index = edge_index_cpu.to(device)
    src, dst = edge_index[0], edge_index[1]


    # We want to add a density feature to each nodes expressing how many neighbours it has in the radius
    # But we use different radii - so we count only the edges within the radius corresponding to that for a human (so 1st element in radii)
    # Essentially for humans this density is just the node degree (minus the edge to previous position) - represents n of objects within the radius
    # For other nodes we only count the nodes within the human - sized radius. Why? if we decide that the train doors should be connected to a very large
    # Area, a density measure would not be local but basically global and wouldn't make much sense

    # UPDATE: I have decided to only do humans in this code section for now, hence the code is refactored

    # 4) Add density and attach it to the node features

    deg = degree(edge_index[0], num_nodes=node_feats.size(0), dtype=torch.float).view(-1,1).to(device) 
    # Note that the tensor (matrix) with edges is of size 2 x n_edges - row 0 is the source nodes and row 1 is the destination nodes
    # We use edges radius and not edge_index because we want the degree of the nodes in the radius graph, not the previous edges
    node_feats = torch.cat([node_feats, deg], dim=1).to(device)

    src, dst = edge_index
    delta_pos = pos[dst] - pos[src]
    
    # 5) Create Python Geometric (a GNN library essentially) Data object
    data = Data(
        x = node_feats,
        edge_index = edge_index,
        edge_attr = delta_pos
    )
    return data, x_pos_unn, y_pos_unn


# Train doors, obstacles and escalators
Now, we will add the nodes corresponding to train doors. They are not given, so we need to make an estimate of when a train arrived at the station.  The idea is to make anchor points at the end of the station, spaced out by 5 meters or so. (We need to find how much that is in pixels). Then, we check how many people are close to each anchor at each timestamp we construct a graph for, and how many people are on the platform. If there is a significantly higher density of people next to the an anchor point than on the platform, we activate that anchor point as a train door edge.

Sidenote: I have also realised that we should construct a graph for each 30 seconds or so to give the network enough context but at the same time to not have a massive blob of a graph. We definitely cannot make a graph of the whole file as it will be a mess.

Sidenote #2: Because making multiple train door nodes offset by time would be very messy, we assume that if a train door is there at the time of making the graph, it was there in the previous 30 seconds (or however often we decide to make the graphs)

In [45]:
dh.fetch_prorail_train_information()
trains_global = dh.train_information
trains_global = trains_global.dropna()
trains_global.head(20)

trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/ehv_train_information_pss.parquet
data fetched. Accessible as <this-object>.train_information


,date,station,train_number,arrival_time,departure_time,door_no,door_x,door_y
39578,2022-01-01,Ehv,832,2022-01-01 10:02:53+01:00,2022-01-01 10:06:09+01:00,1.0,12775.790848,3794.958850
39579,2022-01-01,Ehv,832,2022-01-01 10:02:53+01:00,2022-01-01 10:06:09+01:00,2.0,19865.152225,3699.799637
39580,2022-01-01,Ehv,832,2022-01-01 10:02:53+01:00,2022-01-01 10:06:09+01:00,3.0,39753.360656,3432.843819
39581,2022-01-01,Ehv,832,2022-01-01 10:02:53+01:00,2022-01-01 10:06:09+01:00,4.0,48072.611241,3321.176026
39582,2022-01-01,Ehv,832,2022-01-01 10:02:53+01:00,2022-01-01 10:06:09+01:00,5.0,67070.899838,3066.165441
39583,2022-01-01,Ehv,834,2022-01-01 10:32:55+01:00,2022-01-01 10:36:09+01:00,1.0,-1632.460818,3988.358201
39584,2022-01-01,Ehv,834,2022-01-01 10:32:55+01:00,2022-01-01 10:36:09+01:00,2.0,18255.747613,3721.402384
39585,2022-01-01,Ehv,834,2022-01-01 10:32:55+01:00,2022-01-01 10:36:09+01:00,3.0,25345.108989,3626.243170
39586,2022-01-01,Ehv,834,2022-01-01 10:32:55+01:00,2022-01-01 10:36:09+01:00,4.0,45233.317420,3359.287353
39587,2022-01-01,Ehv,834,2022-01-01 10:32:55+01:00,2022-01-01 10:36:09+01:00,5.0,53072.611241,3254.061932


In [ ]:
def add_doors(
    df,
    trains,
    time,
    radii=RADII,
):

    data, human_x_unn, human_y_unn = human_graph(df, time, radii) #Human graph
    print("Data size: ", data.x.size(0), " edges: ", data.edge_index.size(1))

    human_pos = df[['x_pos', 'y_pos']].values
    human_tree = cKDTree(human_pos)

    A = trains['door_no'].count() # Number of doors

    nodeid = dx = torch.zeros((A, 1)).to(device)
    x_pos_unn = torch.tensor(trains['door_y'].values, dtype=torch.float, device=device).view(A,1) 
    y_pos_unn = torch.tensor(trains['door_x'].values, dtype=torch.float, device=device).view(A,1)

    dx = torch.zeros((A, 1)).to(device)
    dy = torch.zeros((A, 1)).to(device)
    type_onehot = F.one_hot(torch.full((A,), 1, dtype=torch.long), num_classes=6).to(torch.float).to(device)
    coords = np.column_stack((x_pos_unn.cpu().numpy(), y_pos_unn.cpu().numpy()))
    counts = human_tree.query_ball_point(coords, r=radii[1], return_length=True)
    density = torch.tensor(counts, dtype=torch.float, device=device).view(A, 1)
    tau   = sincos_time_embed(time, A, d_tau=8).to(device)     
        
    
    train_nf = torch.cat(   
        [nodeid, x_pos_unn, y_pos_unn, dx, dy, type_onehot, density, tau],
            dim=1
    ).to(device)                                                       

    human_pos_t = torch.cat([human_x_unn, human_y_unn], dim=1)                        
    door_pos_t  = torch.cat([x_pos_unn, y_pos_unn], dim=1)   
    all_pos     = torch.cat([human_pos_t, door_pos_t], dim=0).cpu()
    edge_idx_cpu = radius_graph(all_pos, r=radii[2], loop=False)
    edges_radius = edge_idx_cpu.to(device)

    src, dst = edges_radius
    N = human_pos_t.size(0)
    mask = ((src < N) & (dst >= N)) | ((src >= N) & (dst < N))
    src_cd = src[mask]
    dst_cd = dst[mask]

    full_pos = torch.cat([human_pos_t, door_pos_t], dim=0).to(device)
    delta_cd = full_pos[dst_cd] - full_pos[src_cd]

    new_edge_index = torch.stack([src_cd, dst_cd], dim=0)

    combined_x = torch.cat([data.x, train_nf], dim=0).to(device)
    combined_edge_index = torch.cat([data.edge_index, new_edge_index], dim=1)
    combined_edge_attr  = torch.cat([data.edge_attr, delta_cd], dim=0)
    
    data = Data(
        x          = combined_x,
        edge_index = combined_edge_index,
        edge_attr  = combined_edge_attr
    )
        
    return data, all_pos


We have now added train door nodes. We will now add obstacles, escalators and stairs. Note that these will also be static and hence we will not care about time for them. We will assume they have always existed.

In [ ]:
from scipy.spatial import cKDTree
from torch_geometric.data import Data
from torch_cluster import radius_graph
import torch
import torch.nn.functional as F
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def add_static(tree, time, coords, onehot_idx, radius, num_classes=6):
    N = coords.shape[0]
    assert 0 <= onehot_idx < num_classes

    nodeid = torch.zeros((N, 1)).to(device)
    x = torch.tensor((coords[:, 0] ), dtype=torch.float, device=device).view(N, 1)
    y = torch.tensor((coords[:, 1]), dtype=torch.float, device=device).view(N, 1)
    dx = torch.zeros((N, 1), dtype=torch.float, device=device)
    dy = torch.zeros((N, 1), dtype=torch.float, device=device)
    onehot = F.one_hot(
        torch.full((N,), onehot_idx, dtype=torch.long), num_classes
    ).to(torch.float).to(device)
    counts = tree.query_ball_point(coords, r=radius, return_length=True)
    density = torch.tensor(counts, dtype=torch.float, device=device).view(N, 1)
    tau = sincos_time_embed(time, N, d_tau=8).to(device)

    feats = torch.cat([nodeid, x, y, dx, dy, onehot, density, tau], dim=1)
    return feats


def add_obstacles_stairs(
    df,
    trains,
    time,
    obstacle_ycoords=[17000, 22000],
    obstacle_xcoords=[-3000, -3000],
    stair_ycoords=[0],
    stair_xcoords=[-2000],
    escalator_up_ycoords=[0],
    escalator_up_xcoords=[0],
    escalator_down_ycoords=[0],
    escalator_down_xcoords=[-5000],
    radii=RADII,
):

    data, all_pos_unn = add_doors(df, trains, time)
    human_pos = data.x[:, 1:3].cpu()  
    N_human   = human_pos.size(0)

    tree = cKDTree(data.x[:, 1:3].cpu().cpu().numpy())

    all_feats       = [data.x]
    all_edge_index  = [data.edge_index]
    all_edge_attr   = [data.edge_attr]

    node_offset = N_human

    specs = [
        (np.column_stack((obstacle_xcoords, obstacle_ycoords)),    2),
        (np.column_stack((stair_xcoords,    stair_ycoords)),       3),
        (np.column_stack((escalator_up_xcoords,   escalator_up_ycoords)),   4),
        (np.column_stack((escalator_down_xcoords, escalator_down_ycoords)), 5),
    ]

    for coords, onehot_idx in specs:
        
        if coords.size == 0:
            continue 

        rad = radii[onehot_idx]

        feats_static = add_static(tree, time, coords, onehot_idx, rad)
        A = feats_static.size(0)      # number of static nodes

        static_pos = torch.from_numpy(coords).float()  
        all_pos_cpu = torch.cat([all_pos_unn, static_pos], dim=0)

        edge_idx_cpu = radius_graph(all_pos_cpu, r=rad, loop=False)
        src_cpu, dst_cpu = edge_idx_cpu

        mask = (
            ((src_cpu < N_human) & (dst_cpu >= N_human)) |
            ((src_cpu >= N_human) & (dst_cpu < N_human))
        )
        src_cd = src_cpu[mask]
        dst_cd = dst_cpu[mask]

        def to_global(idx):
            human_mask = idx < N_human
            static_mask = ~human_mask
            out = idx.clone()
            out[static_mask] = (idx[static_mask] - N_human) + node_offset
            return out
        src_glob = to_global(src_cd)
        dst_glob = to_global(dst_cd)

        full_pos = torch.cat([human_pos, static_pos], dim=0)
        delta_cd = full_pos[dst_cd] - full_pos[src_cd]

        new_edge_index = torch.stack([src_glob, dst_glob], dim=0).to(device)
        new_edge_attr  = delta_cd.to(device)

        all_feats.append(feats_static.to(device))
        all_edge_index.append(new_edge_index)
        all_edge_attr.append(new_edge_attr)

        node_offset += A

    data.x = torch.cat(all_feats, dim=0)
    edge_index = torch.cat(all_edge_index, dim=1)
    edge_attr  = torch.cat(all_edge_attr, dim=0)

    edge_index, edge_attr = add_self_loops(
        edge_index,
        edge_attr=edge_attr,
        fill_value=0.0,
        num_nodes=data.x.size(0)
    )

    data.edge_index = edge_index
    data.edge_attr  = edge_attr
    return data


The following code was used to create our weekly datasets:

In [ ]:
import os
import time
import torch
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone
from pathlib import Path
from torch_geometric.nn import radius_graph
import torch.nn.functional as F
from torch_geometric.data import Data
from zoneinfo import ZoneInfo

def process_block_sequences(block_start: datetime, trains_global: pd.DataFrame,
                            dh) -> list:
    """
    Process sequences of 10 snapshots (1 per second) starting from block_start for one day,
    keeping only people present in all 10 frames. Returns list of sequences.
    """
    date_str = block_start.strftime('%Y%m%d')

    # Load and concatenate all hourly files for that day
    dfs = []
    for hour in range(24):
        path = f"ehv/platform2.1/{date_str}/EHV_Platform2.1_{date_str}{hour:02}_trajectorie.parquet"
        try:
            dh.fetch_prorail_data_from_path(path)
            dfs.append(dh.df.copy())
        except Exception:
            continue
    if not dfs:
        print(f"No data for {date_str}, skipping.")
        return []

    df_day = pd.concat(dfs, ignore_index=True)
    df_day = add_dx_dy_ntype_city(df_day)
    df_day['datetime'] = (
        pd.to_datetime(df_day.date_time_utc, unit='ms')
          .dt.tz_localize('UTC')
          .dt.tz_convert('Europe/Berlin')
    )
    df_day = df_day[df_day['datetime'].dt.microsecond == 0]
    timestamps = sorted(df_day['datetime'].unique())

    # Normalize tau bounds for the day
    min_time = df_day['date_time_utc'].min()
    max_time = df_day['date_time_utc'].max()

    sequences = []
    seq_count = 0
    for i in range(0, len(timestamps) - 9, 10):
        window = timestamps[i: i+10]

        #below we filter for graphs which are present across all 10 elements in our sequence
        df_window = df_day[df_day['datetime'].isin(window)]
        counts = df_window.groupby('tracked_object')['datetime'].nunique()
        persistent_ids = counts[counts == 10].index
        df_persistent = df_window[df_window['tracked_object'].isin(persistent_ids)]
        if df_persistent.empty:
            continue

        try:
            graphs = []
            for ts in window:
                slice_df = df_persistent[df_persistent['datetime'] == ts]
                curr_ms = int(ts.timestamp() * 1000)
                tau = (curr_ms - min_time) / (max_time - min_time) #normalize time
                trains_local = trains_global[
                    (trains_global['arrival_time'] <= ts) &
                    (trains_global['departure_time'] >= ts)
                ] #extract trains
                graph = add_obstacles_stairs(slice_df, trains_local, tau,
                                            radii=RADII) #create graph
                graphs.append(graph)
            sequences.append(graphs)
            seq_count += 1
            print(f"Collected seq {seq_count} for {date_str} at {window[0].strftime('%H:%M:%S')}") #debug prints
        except Exception as e:
            print(f"Skipping from {window[0].strftime('%H:%M:%S')} due to: {e}") #skip in case of error

    return sequences


def build_weekly_sequences(start_date: datetime, end_date: datetime,
                           trains_global: pd.DataFrame, dh,
                           output_dir: Path):
    """
    We make one file per week with 10 size sequences for the 10 seconds. 
    """
    current = start_date
    one_day = timedelta(days=1)
    week_num = 1

    while current <= end_date:
        week_start = current
        week_end = min(current + timedelta(days=6), end_date)
        weekly_sequences = []
        print(f"Processing Week {week_num}: {week_start.date()} to {week_end.date()}")

        # Collect sequences for each day in the week
        day = week_start
        while day <= week_end:
            daily_seqs = process_block_sequences(day, trains_global, dh)
            weekly_sequences.extend(daily_seqs)
            day += one_day

        # Save weekly file
        week_str = f"{week_start.strftime('%Y%m%d')}_{week_end.strftime('%Y%m%d')}"
        out_file = output_dir / f"sequences_week_{week_str}.pt"
        torch.save(weekly_sequences, out_file)
        print(f"Saved {len(weekly_sequences)} sequences for Week {week_num} to {out_file}")

        #next week
        current = week_end + one_day
        week_num += 1


def main():
    # Initialize data handler and train info
    dh = SociophysicsDataHandler()
    dh.fetch_prorail_train_information()
    trains_global = dh.train_information.dropna()
    trains_global['arrival_time'] = (
        pd.to_datetime(trains_global['arrival_time'], utc=True)
          .dt.tz_convert('Europe/Berlin')
    )
    trains_global['departure_time'] = (
        pd.to_datetime(trains_global['departure_time'], utc=True)
          .dt.tz_convert('Europe/Berlin')
    )

    # Define start and end dates
    start_date = datetime(2022, 1, 15, tzinfo=ZoneInfo("Europe/Berlin"))
    end_date   = datetime(2022, 12, 31, tzinfo=ZoneInfo("Europe/Berlin"))

    # Prepare output directory
    output_dir = Path(r"graphsequences")
    output_dir.mkdir(parents=True, exist_ok=True)

    # Build weekly sequence files
    build_weekly_sequences(
        start_date,
        end_date,
        trains_global,
        dh,
        output_dir
    )

main()


The following code was used to create graph snapshots for autoregressive unroll:

In [ ]:
import os
import time
import torch
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone
from pathlib import Path
from zoneinfo import ZoneInfo
import matplotlib.pyplot as plt
import networkx as nx
from torch_geometric.utils import to_networkx

def generate_single_snapshots_block(start_ts: datetime,
                                    trains_global: pd.DataFrame,
                                    dh,
                                    output_dir: Path,
                                    num_snapshots: int = 10) -> None: # Returns none since we just save sequences
    output_dir.mkdir(parents=True, exist_ok=True)

    # Load and preprocess full day data
    date_str = start_ts.strftime('%Y%m%d')
    dfs = []
    for hr in range(24):
        path = f"ehv/platform2.1/{date_str}/EHV_Platform2.1_{date_str}{hr:02}_trajectorie.parquet"
        try:
            dh.fetch_prorail_data_from_path(path)
            dfs.append(dh.df.copy())
        except Exception:
            continue #sometimes loading would not work and crash my dataset creation hence this was added
    if not dfs:
        raise FileNotFoundError(f"No data for {date_str}")

    df_day = pd.concat(dfs, ignore_index=True)
    df_day = add_dx_dy_ntype_city(df_day)
    df_day['datetime'] = (
        pd.to_datetime(df_day.date_time_utc, unit='ms')
          .dt.tz_localize('UTC')
          .dt.tz_convert('Europe/Berlin')
    )
    df_day = df_day[df_day['datetime'].dt.microsecond == 0]

    min_time = df_day['date_time_utc'].min()
    max_time = df_day['date_time_utc'].max()

    timestamps = [start_ts + timedelta(seconds=i) for i in range(num_snapshots)]

    # Take only nodes present on all 10 elements in a sequence
    df_window = df_day[df_day['datetime'].isin(timestamps)]
    counts = df_window.groupby('tracked_object')['datetime'].nunique()
    persistent_ids = counts[counts == num_snapshots].index
    df_persistent = df_window[df_window['tracked_object'].isin(persistent_ids)]

    if df_persistent.empty:
        print(f"No consistent nodes across all snapshots starting at {start_ts}")
        return

    for i, ts in enumerate(timestamps):
        slice_df = df_persistent[df_persistent['datetime'] == ts]
        if slice_df.empty:
            print(f"No data at {ts}, skipping snapshot {i}")
            continue

        curr_ms = int(ts.timestamp() * 1000)
        tau = (curr_ms - min_time) / (max_time - min_time)
        trains_local = trains_global[
            (trains_global['arrival_time'] <= ts) &
            (trains_global['departure_time'] >= ts)
        ]

        try:
            graph = add_obstacles_stairs(slice_df, trains_local, tau, radii=RADII)
            ts_str = ts.strftime('%Y%m%d_%H%M%S')
            out_graph = output_dir / f"snapshot_{ts_str}.pt"
            torch.save(graph, out_graph)
            print(f"Saved snapshot {i+1}/{num_snapshots} at {ts} to {out_graph}") #debug print

            # Plot first snapshot for debug purposes
            if i == 0:
                nx_graph = to_networkx(graph, to_undirected=False)
                pos = {n: (graph.x[n][1].item(), graph.x[n][2].item())
                       for n in range(graph.num_nodes)}

                plt.figure(figsize=(10, 8))
                nx.draw(nx_graph, pos, with_labels=True,
                        node_size=300, node_color='skyblue',
                        edge_color='gray', arrows=True)
                self_loops = [(u, v) for u, v in nx_graph.edges() if u == v]
                nx.draw_networkx_edges(
                    nx_graph, pos, edgelist=self_loops,
                    edge_color='red', width=2.0,
                    connectionstyle='arc3,rad=0.4'
                )
                plot_file = output_dir / f"snapshot_{ts_str}.png"
                plt.title(f"Graph Snapshot at {ts}")
                plt.savefig(plot_file)
                plt.close()
                print(f"Saved image of first snapshot {plot_file}")

        except Exception as e:
            print(f"Error at {ts}: {e}")


def main():
    # Initialize handler and train info
    dh = SociophysicsDataHandler()
    dh.fetch_prorail_train_information()
    trains_global = dh.train_information.dropna()
    trains_global['arrival_time'] = pd.to_datetime(
        trains_global['arrival_time'], utc=True
    ).dt.tz_convert('Europe/Berlin')
    trains_global['departure_time'] = pd.to_datetime(
        trains_global['departure_time'], utc=True
    ).dt.tz_convert('Europe/Berlin')

    # Starting timestamp
    start_ts = datetime(2023, 7, 26, 18, 12, 52, tzinfo=ZoneInfo("Europe/Berlin"))
    output_dir = Path(r"C:\Users\karol\Desktop\FLOWCBLTHESEUSCONVNOGRUWITHID")

    generate_single_snapshots_block(
        start_ts, trains_global, dh, output_dir, num_snapshots=10
    )

main()


trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/ehv_train_information_pss.parquet
data fetched. Accessible as <this-object>.train_information
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20230726/EHV_Platform2.1_2023072600_trajectorie.parquet


C:\Users\karol\AppData\Local\Temp\ipykernel_21160\3263791699.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trains_global['arrival_time'] = pd.to_datetime(
C:\Users\karol\AppData\Local\Temp\ipykernel_21160\3263791699.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trains_global['departure_time'] = pd.to_datetime(


data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20230726/EHV_Platform2.1_2023072601_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20230726/EHV_Platform2.1_2023072602_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20230726/EHV_Platform2.1_2023072603_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20230726/EHV_Platform2.1_2023072604_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20230726/EHV_Platform2.1_2023072605_trajectorie.parquet
data fetched. Accessible as <this-object>.df
trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20230726/EHV_Platform2.1

# ----------------- IMPORT ---------------

The previous part of our code dealt with creating our dataset. We now move on to making our model by defining our DataLoader. This object loads data into our GNN. 

In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Batch
from torch_geometric.nn import GATConv, GCNConv
import contextlib
import cProfile
import pstats

In [ ]:
from torch.utils.data import Dataset
from torch_geometric.data import Batch
import torch
import os

def graph_sequence_collate(batch):
    """
    Function to make batches from sequences
    """
    seq_lists, tgt_list = zip(*batch)
    seq_len = len(seq_lists[0])

    batched_seq = []
    for t in range(seq_len):
        graphs_at_t = [seq_lists[i][t] for i in range(len(batch))]
        batched = Batch.from_data_list(graphs_at_t, exclude_keys=['y'])
        batched_seq.append(batched)

    batched_target = Batch.from_data_list(tgt_list)
    return batched_seq, batched_target

WINDOW = 5

#Custom dataset class was necessary
class GraphSequenceDataset(Dataset):
    def __init__(self, graph_dir, file_list):
        self.pairs = []
        for fn in file_list:
            seqs = torch.load(os.path.join(graph_dir, fn), map_location='cpu', weights_only=False)
            for seq in seqs:
                n0 = seq[0].num_nodes
                if n0 < 5 or any(g.num_nodes != n0 for g in seq):
                    continue #if somehow the number of nodes is not constant despite the dataset theoretically not creating such sequences, drop the sequence

                # build sliding windows
                for i in range(len(seq) - WINDOW):
                    inp = seq[i : i + WINDOW]
                    tgt = seq[i + WINDOW].clone()

                    # compute displacement
                    prev = inp[-1]
                    disp = tgt.x[:, 1:3] - prev.x[:, 1:3]
                    tgt.y = disp

                    self.pairs.append((inp, tgt))

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        return self.pairs[idx]




from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn

#Our model
class ThomasGNN(nn.Module):
    def __init__(self, in_feats, gat_hidden, mlp_hidden):
        super().__init__()
        #Convolution on each of the 5 input graphs
        self.gat1 = GCNConv(
            in_channels=in_feats,
            out_channels=gat_hidden
        )

        self.gat2 = GCNConv(    
            in_channels=gat_hidden,
            out_channels=gat_hidden
        )
        # MLP over concatenated embeddings of the 5 input graphs
        self.mlp = nn.Sequential(
            nn.Linear(gat_hidden * WINDOW, mlp_hidden),
            nn.LeakyReLU(),
            nn.Linear(mlp_hidden, 2)
        )

    def forward(self, batched_seq):
        embeds = []
        for batch in batched_seq:
            x, edge_index = batch.x, batch.edge_index
            x = F.elu(self.gat1(x, edge_index))  
            x = self.gat2(x, edge_index)          
            embeds.append(x)

        x_cat = torch.cat(embeds, dim=1)
        preds = self.mlp(x_cat)                 
        return preds



Now, let's make our model!

In [ ]:
def run_epoch(model, loader, optimizer, device, train=True):
    model.train() if train else model.eval()
    total_loss = 0.0
    ctx = torch.no_grad() if not train else contextlib.nullcontext()

    with ctx:

        for batch_idx, (batched_seq, batched_target) in enumerate(loader):
            batched_seq    = [b.to(device, non_blocking=True) for b in batched_seq]
            batched_target = batched_target.to(device, non_blocking=True)

            #Compute displacement
            prev      = batched_seq[-1]
            pos_prev  = prev.x[:, 1:3]             
            pos_curr  = batched_target.x[:, 1:3]  
            disp_true = pos_curr - pos_prev        

            if train:
                optimizer.zero_grad()
            preds = model(batched_seq, batched_target)

            #loss only on human nodes
            human_mask = (batched_target.x[:, 5] == 1)
            if batch_idx % 10000 == 0:
                print("preds: ", preds[human_mask], "disp true", disp_true[human_mask])
            sq_err = (preds - disp_true).pow(2).sum(dim=1)        

            #Weighted loss funciton
            mag    = disp_true.norm(dim=1)                        
            alpha  = 1.0                                          
            beta   = 2.0                                          
            weights = alpha + beta * mag                          

            w = weights[human_mask]                              
            e = sq_err   [human_mask]
            
            loss = (w * e).sum() / w.sum()

            if train:
                loss.backward()
                optimizer.step()

            total_loss += loss.item()
            if batch_idx % 1000 == 0:
                print(f"[{'train' if train else 'val'}] "
                    f"Batch {batch_idx:03d} | Loss: {loss:.4f}  "
                    f"({human_mask.sum().item()} humans)")
                
            if (batch_idx % 100000 == 0) & (batch_idx > 0): 
                out = f"model_100000.pt" #periodically save the model; program would sometimes crash while training and progress would be lost
                torch.save({k: v.cpu().clone() for k, v in model.state_dict().items()}, out)


    return total_loss / len(loader)



import os
import torch
import contextlib
import torch.nn as nn
import cProfile, pstats
from torch.utils.data import Dataset, DataLoader
from torch_geometric.data import Batch

torch.autograd.set_detect_anomaly(True) #For debug purposes

def main():
    """
    Load each dataset file containing graphs for the given week in chronological order, train the network.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Directory with monthly graph files
    graphs_dir = r"graphsequences"
    all_files = sorted(
        [f for f in os.listdir(graphs_dir) if f.endswith(".pt")],
    )
    if not all_files:
        raise RuntimeError(f"No .pt files found in {graphs_dir}")

    # Hyperparameters
    batch_size = 8
    epochs_per_month = 1
    learning_rate = 5e-3

    # Instantiate a single model once
    model = ThomasGNN(
        in_feats=20,
        gat_hidden=32,
        gat_heads=4,
        mlp_hidden=32,
    ).to(device)

    # Load existing checkpoint if present
    existing_ckpts = [f for f in os.listdir('.') if f.startswith('model_') and f.endswith('.pt')] #models are saved with this template and can therefore be loaded
    if existing_ckpts:
        # Pick the latest by name (largest YYYYMM)
        latest = sorted(existing_ckpts)[-1]
        print(f"Loading existing checkpoint: {latest}")
        state = torch.load(latest, map_location='cpu', weights_only=False)
        model.load_state_dict(state)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=3
    )

    # Training loop over months
    for month_fn in all_files:
        month_str = month_fn.split("_")[1].split(".")[0]
        print(f"\n=== Processing Month {month_str} ({month_fn}) ===")

        # Build dataset for this month
        ds = GraphSequenceDataset(
            graph_dir=graphs_dir,
            file_list=[month_fn],
        )
        N = len(ds)
        if N <= 0:
            print(f"Skipping {month_fn}: not enough data")
            continue

        # Split 90/10
        split = int(0.9 * N)
        train_ds = torch.utils.data.Subset(ds, list(range(split)))
        val_ds   = torch.utils.data.Subset(ds, list(range(split, N)))

        train_loader = DataLoader(
            train_ds, batch_size=batch_size, shuffle=True,
            collate_fn=graph_sequence_collate, pin_memory=True
        )
        val_loader = DataLoader(
            val_ds, batch_size=batch_size, shuffle=False,
            collate_fn=graph_sequence_collate, pin_memory=True
        )

        best_val = float('inf')
        # Train for this month
        for epoch in range(1, epochs_per_month + 1):
            print(f"-- [Month {month_str}] Epoch {epoch}/{epochs_per_month} --")
            train_loss = run_epoch(model, train_loader, optimizer, device, train=True)
            val_loss   = run_epoch(model, val_loader,   optimizer, device, train=False)
            scheduler.step(val_loss)
            print(f"    → train={train_loss:.4f}   val={val_loss:.4f}")
            if val_loss < best_val:
                best_val = val_loss
                # Save checkpoint
                out = f"model_{month_str}.pt"
                torch.save({k: v.cpu().clone() for k, v in model.state_dict().items()}, out)
                print(f"    ↪ New best val: {val_loss:.4f}, saved to {out}")

    print("\nTraining complete. Final model saved as latest 'model_YYYYMM.pt'.")


main()

Loading existing checkpoint: model_week.pt

=== Processing Month week (sequences_week_20220115_20220121.pt) ===
-- [Month week] Epoch 1/1 --
preds:  tensor([[  114.0777,    58.4473],
        [  114.0777,    58.4473],
        [  105.3893,    74.0436],
        [  114.8157,   109.6628],
        [  103.9600,    64.3489],
        [   78.2239,   408.0450],
        [   46.9439,   120.8828],
        [   78.2239,   408.0450],
        [  100.1101,    52.4426],
        [  103.2700,    63.7383],
        [   50.9467,    46.8567],
        [   49.2432,   117.9175],
        [   47.9481,   245.1605],
        [  111.8859,   434.7928],
        [  127.9607,   979.1915],
        [   47.9712,   184.6017],
        [  107.3091,    48.5021],
        [   52.3414,   205.6376],
        [   78.3738,  -722.8117],
        [  107.1838,   -11.8222],
        [  146.0721,  1473.2473],
        [  145.2598,  1503.8427],
        [   51.3108,   156.4978],
        [  151.4967,  1510.8473],
        [  166.4377,  1545.1991],
 

RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

# Sample predictions of our model in a CSV file:

In [ ]:
import os
import pandas as pd
import torch
import torch.nn.functional as F
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo
from torch_geometric.data import Data


WINDOW = 5 

def load_sequence_snapshots(snapshot_dir):
    """
    Return sorted list of snapshot file paths
    """
    fps = [os.path.join(snapshot_dir, f)
           for f in os.listdir(snapshot_dir)
           if (f.endswith('.pt') & f.startswith('snapshot'))]
    return sorted(fps)



def load_checkpoint(model, ckpt_path, device):
    """
    Load model from a file and move to device.
    """
    state = torch.load(ckpt_path, map_location='cpu', weights_only=False)
    model.load_state_dict(state)
    model.to(device)


def unroll_autoregressive(
    model,
    snapshot_dir: str,
    output_dir: str,
    device,
    collate_fn,
    start_ts: datetime,
    dh: 'SociophysicsDataHandler'
):
    """
    Fetch true train info, predict displacement from a snapshot created from the snapshot function defined in an earlier cell, 
    compute predictions from that shapshot to perform an autoregressive unroll.
    """
    os.makedirs(output_dir, exist_ok=True)
    model.eval()

    # Fetch and prepare data at the specified timestamp - this gives us the ground-truth positions
    date_str = start_ts.strftime('%Y%m%d')
    hour = start_ts.strftime('%H')
    path = f"ehv/platform2.1/{date_str}/EHV_Platform2.1_{date_str}{hour:02}_trajectorie.parquet"
    dh.fetch_prorail_data_from_path(path)
    # Load full day trajectories via handler
    df_day = dh.df.copy()
    df_day['datetime'] = (
        pd.to_datetime(df_day.date_time_utc, unit='ms')
          .dt.tz_localize('UTC')
          .dt.tz_convert('Europe/Berlin')
    )
    # Slice the exact second and the next second
    t0 = df_day[df_day['datetime'] == start_ts]
    t1 = df_day[df_day['datetime'] == (start_ts + timedelta(seconds=1))]

    # Build a mapping: (node id) -> (true position at t0 and t1)
    true_t0 = t0.set_index('tracked_object')[['x_pos', 'y_pos']]
    true_t1 = t1.set_index('tracked_object')[['x_pos', 'y_pos']]

    # 2. Load initial buffer of true snapshots from disk
    buffer = [torch.load(fp, map_location=device, weights_only=False)
              for fp in snapshot_dir[:WINDOW]]

    step = 0
    while step < 60:
        batched_seq, _ = collate_fn([(buffer, buffer[-1])])
        batched_seq = [b.to(device) for b in batched_seq]

        #Run model on batched sequences
        embeds = []
        for b in batched_seq:
            x, edge_index = b.x, b.edge_index
            h = F.elu(model.gat1(x, edge_index))
            h = model.gat2(h, edge_index)
            embeds.append(h)

        x_cat = torch.cat(embeds, dim=1)
        preds = model.mlp(x_cat).detach().cpu()  # shape [N,2]

        #Columns for csv
        records = []
        last = buffer[-1]
        curr_x = last.x[:,1:3].cpu().numpy()
        feature_type = last.x[:,5].cpu().numpy() 

        # Compute predictions and errors for each node
        for node_idx in range(curr_x.shape[0]):
            if feature_type[node_idx] == 0:
                # skip static nodes
                continue

            node_ids = last.x[:,0].cpu().numpy()
            node_id = node_ids[node_idx]
            # current pos
            cx, cy = curr_x[node_idx]
            # predicted displacement
            dx, dy = preds[node_idx]
            px, py = cx + dx, cy + dy
            # true positions if available
            if node_id in true_t0.index and node_id in true_t1.index:
                tx0, ty0 = true_t0.loc[node_id]
                tx1, ty1 = true_t1.loc[node_id]
            else:
                tx0 = ty0 = tx1 = ty1 = float('nan')

            # displacement error to compute ADE
            err = ((tx1 - px)**2 + (ty1 - py)**2)**0.5

            records.append({
                'node_id': node_id,
                'node_type': int(feature_type[node_idx]),
                'curr_x': cx,
                'curr_y': cy,
                'pred_x': px,
                'pred_y': py,
                'true_x': tx1,
                'true_y': ty1,
                'error': err
            })

        # Save to csv
        df_out = pd.DataFrame(records)
        out_csv = os.path.join(output_dir, f'unroll_step_{step}.csv')
        df_out.to_csv(out_csv, index=False)

        disp_pred = preds.clone()
        new_x = last.x.clone().cpu()
        # Zero-out predictions for non human nodes since they cannot move
        disp_pred[feature_type == 0] = 0
        new_positions = new_x[:,1:3].cpu() + disp_pred
        new_x[:,1:3] = new_positions
        new_data = Data(x=new_x.to(device), edge_index=last.edge_index)

        buffer.pop(0)
        buffer.append(new_data)
        step += 1

    print(f"Autoregressive unroll completed for {start_ts}")
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ThomasGNN(in_feats=20, gat_hidden=32, gat_heads=4, mlp_hidden=32).to(device)
load_checkpoint(model, 'modelidkbackup.pt', device)

!pip install -q sociophysicsDataHandler
from sociophysicsDataHandler import SociophysicsDataHandler
dh = SociophysicsDataHandler()

snapshot_dir = r"C:\Users\karol\Desktop\FLOWCBLTHESEUSCONVNOGRUWITHID"
fps = load_sequence_snapshots(snapshot_dir)
start_ts = datetime(2023, 7, 28, 18, 29, tzinfo=ZoneInfo("Europe/Berlin"))
unroll_autoregressive(model, fps, 'plots_output_unroll', device, graph_sequence_collate, start_ts, dh, )

You should consider upgrading via the 'C:\Users\karol\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


trying to fetch: /storage/surfsara/ProRail_USE_LL_data/ehv/platform2.1/20230728/EHV_Platform2.1_2023072818_trajectorie.parquet
data fetched. Accessible as <this-object>.df
Autoregressive unroll completed for 2023-07-28 18:29:00+02:00
